In [20]:
import numpy as np
# import pandas as pd
# import sys, re, textwrap
# import pickle
# import lzma

In [ ]:
# import time
# import math

In [1]:
# import os
# import random
# from string import ascii_letters

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import random_split
# from unidecode import unidecode

/home/dima/miniconda3/envs/ctpesto/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.dataset import SeqDataset, get_stat_from_dataset
from model import RNN
from torch.utils.data import DataLoader

In [3]:
from src.data_encoding import encode_res, all_resnames,encode_location, selected_locations

# Setup

In [4]:
_ = torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cuda')

In [49]:
def get_stat_from_dataset1(seq_dataset):
    all_locations = []
    for i in seq_dataset:
        j = i[1].numpy()
        all_locations.append(j)
    summary = np.sum(np.array(all_locations), axis=0)
    report = ""
    for i, value in enumerate(summary):
        stri = f'{selected_locations[i]}: {str(int(value))}\n'
        report += stri
    return report


# Defs

In [6]:
nres = 1024

In [7]:
dataset = SeqDataset("../data/data_seq_locations.xz", nres)

In [21]:
get_stat_from_dataset1(dataset)

array([31341.,  5065., 12093.,  6399.])

In [23]:
print("length of the dataset is:", len(dataset))

length of the dataset is: 42511


In [24]:
train_dataset, test_dataset = random_split(dataset, [len(dataset)-2500, 2500])

In [50]:

print(get_stat_from_dataset1(test_dataset))



membrane: 1845
cytoplasm: 277
mitochondrion: 736
nucleus: 380



## Model

In [ ]:
from torch.autograd import Variable
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, device):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, 
                            batch_first=True, dropout=0.1)
#         self.gru = nn.GRU(input_size, self.hidden_size, self.num_layers, batch_first=True, dropout=0.2)
#         self.fc1 = nn.Linear(hidden_size, int(hidden_size/2))
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(int(hidden_size/2), int(hidden_size/2))
#         self.fc3 = nn.Linear(int(hidden_size/2), num_classes)
        self.hidden2tag = nn.Linear(self.hidden_size, num_classes)
        self.device = device
    
    def forward(self, x):
        x = x.float()
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).float()).to(self.device)
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size).float()).to(self.device)
        out, _ = self.lstm(x, (h0,c0)) 
#         out = self.relu(self.fc1(out[:, -1, :]))
#         out = self.relu(self.fc2(out))
#         out = self.fc3(out) 
        tag_space = self.hidden2tag(out[:, -1, :])
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
n_letters = len(all_resnames)
n_categories = len(selected_locations)
learning_rate = 0.0002

In [ ]:
model = RNN(nres, 64, 4, n_categories, device=device)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
model.parameters

### test the model


In [ ]:
dataloader = DataLoader(train_dataset,batch_size=12)
X = next(iter(dataloader))[0]

In [ ]:
X = X.to(device)
X.shape

In [ ]:
model(X)

## Train the model

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, Y) in enumerate(dataloader):
        # Compute prediction and loss
        x = X.to(device)
        y = Y.to(device)
        pred = model(x)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, Y in dataloader:
            x = X.to(device)
            y = Y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
#     correct /= size
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(f"Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
train_dataloader = DataLoader(train_dataset,batch_size=128)
test_dataloader = DataLoader(test_dataset,batch_size=256)

epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
dataloader = DataLoader(train_dataset,batch_size=20)
x,y = next(iter(dataloader))
x = x.to(device)

In [ ]:
with torch.no_grad():
    y_pred = model(x)
y_pred
a = torch.rand(2,3)

In [ ]:
b = nn.Softmax()
b(y_pred).round()

In [ ]:
y